# DataFrame 을 melt -> pivot -> melt 로 변형시키기
* [kaggle참고](https://www.kaggle.com/residentmario/renaming-and-combining)

In [1]:
import pandas as pd

* DataFrame 생성

In [2]:
df = pd.DataFrame({'A' : {0: 'a', 1: 'b', 2: 'c'},
                   'B' : {0: 1, 1: 3, 2: 5},
                   'C' : {0: 2, 1: 4, 2: 6} })
df

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


## melt()
* id_vars = Column(s) to use as identifier variables
* value_vars = Column(s) to unpivot. If not specified, uses all columns that are not set as `id_vars`.

In [3]:
pd.melt(df, id_vars=['A'], value_vars=['B'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5


In [4]:
pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])

,A,variable,value
0,a,B,1
1,b,B,3
2,c,B,5
3,a,C,2
4,b,C,4
5,c,C,6


* melt 가 되면서 variable index밑으로 columns 값 A, B, C 가 녹아 들어가고,
* 각 columns 의 값은 value column 아래 값으로 **<u>순서대로</u>** 들어간다.

In [5]:
test_df = pd.melt(df, value_vars=['A', 'B', 'C']).rename(columns={'variable':'var', 'value':'val'})

In [6]:
test_df

,var,val
0,A,a
1,A,b
2,A,c
3,B,1
4,B,3
5,B,5
6,C,2
7,C,4
8,C,6


***

## melt 한 DataFrame 을 원상복구
* 컬럼값으로 들어갈 'A','B','C'를 온전히 합치기 위해서 key 컬럼을 만듦
* key 컬럼은 var, val에 중복되지 않아야한다. (사실 이부분은... 좀 거지같아서 정리는 안됨)
* 호기심에 시작한게 이 사단을 일으킴(2-3시간 날린듯...)

In [7]:
test_df['key'] = ['x','y','z']*3

In [8]:
test_df

,var,val,key
0,A,a,x
1,A,b,y
2,A,c,z
3,B,1,x
4,B,3,y
5,B,5,z
6,C,2,x
7,C,4,y
8,C,6,z


In [9]:
test_df1 = test_df.pivot(index='key', columns='var',
                        values='val').reset_index().rename_axis('', axis='columns')

In [10]:
test_df1.drop(columns='key')

,A,B,C
0,a,1,2
1,b,3,4
2,c,5,6


# pivot

* DataFram 생성

In [11]:
df2 = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
                    'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                    'baz': [1, 2, 3, 4, 5, 6]})
df2

,foo,bar,baz
0,one,A,1
1,one,B,2
2,one,C,3
3,two,A,4
4,two,B,5
5,two,C,6


## pivot

In [12]:
df3 = df2.pivot(index='foo', columns='bar', values='baz').reset_index()
df3

bar,foo,A,B,C
0,one,1,2,3
1,two,4,5,6


## melt

In [13]:
df3.melt(id_vars=['foo'],
         value_vars=['A', 'B', 'C']).sort_values(['foo', 'bar']).rename(columns={'value':'baz'})

,foo,bar,baz
0,one,A,1
2,one,B,2
4,one,C,3
1,two,A,4
3,two,B,5
5,two,C,6


# 결론 pivot -> melt & melt -> pivot  columns 명이 망가짐 ㅠㅠ